In [15]:
import regex

In [16]:
cat_pattern = open("categories.txt").read()
cat_pattern

'(?:\\p{Grapheme_Cluster_Break=CR}\\p{Grapheme_Cluster_Break=LF}|\\p{Grapheme_Cluster_Break=CR}|\\p{Grapheme_Cluster_Break=LF})|\\p{Grapheme_Cluster_Break=Control}|\\p{Grapheme_Cluster_Break=Prepend}*(?:(?:\\p{Grapheme_Cluster_Break=L}*(?:\\p{Grapheme_Cluster_Break=V}+|\\p{Grapheme_Cluster_Break=LV}\\p{Grapheme_Cluster_Break=V}*|\\p{Grapheme_Cluster_Break=LVT})\\p{Grapheme_Cluster_Break=T}*|\\p{Grapheme_Cluster_Break=L}+|\\p{Grapheme_Cluster_Break=T}+)|(?:\\p{Grapheme_Cluster_Break=RI}\\p{Grapheme_Cluster_Break=RI})|(?:\\p{Extended_Pictographic}(?:\\p{Grapheme_Cluster_Break=Extend}*\\p{Grapheme_Cluster_Break=ZWJ}\\p{Extended_Pictographic})*)|(?:\\p{InCB=Consonant}(?:(?:\\p{InCB=Extend}|\\p{InCB=Linker})*\\p{InCB=Linker}(?:\\p{InCB=Extend}|\\p{InCB=Linker})*\\p{InCB=Consonant})+)|[^\\p{Grapheme_Cluster_Break=Control}\\p{Grapheme_Cluster_Break=CR}\\p{Grapheme_Cluster_Break=LF}])(?:\\p{Grapheme_Cluster_Break=Extend}|\\p{Grapheme_Cluster_Break=ZWJ}|\\p{Grapheme_Cluster_Break=SpacingMark})*

In [17]:
categories = [
    # this is special cased because negated sets don't play well with variable length character sets
    "[^\\p{Grapheme_Cluster_Break=Control}\\p{Grapheme_Cluster_Break=CR}\\p{Grapheme_Cluster_Break=LF}]",
    # normal classes below
    "\\p{Grapheme_Cluster_Break=CR}",
    "\\p{Grapheme_Cluster_Break=LF}",
    "\\p{Grapheme_Cluster_Break=Control}",
    "\\p{Grapheme_Cluster_Break=Prepend}",
    "\\p{Grapheme_Cluster_Break=L}",
    "\\p{Grapheme_Cluster_Break=V}",
    "\\p{Grapheme_Cluster_Break=LV}",
    "\\p{Grapheme_Cluster_Break=LVT}",
    "\\p{Grapheme_Cluster_Break=T}",
    "\\p{Grapheme_Cluster_Break=RI}",
    "\\p{Extended_Pictographic}",
    "\\p{Grapheme_Cluster_Break=Extend}",
    "\\p{Grapheme_Cluster_Break=ZWJ}",
    "\\p{InCB=Consonant}",
    "\\p{InCB=Extend}",
    "\\p{InCB=Linker}",
    "\\p{Grapheme_Cluster_Break=SpacingMark}",
]

In [18]:
cat_chars: dict[str, list[str]] = {}
for category in categories:
    chars = []
    pat = regex.compile(category)
    
    lim = 0x110000 * 1
    for i in range(lim):
        try:
            chr(i).encode("utf-8")
            if regex.match(pat, chr(i)):
                chars.append(chr(i))
        except:
            pass
    cat_chars[category] = chars

In [19]:
{key: len(value) for key, value in cat_chars.items()}

{'[^\\p{Grapheme_Cluster_Break=Control}\\p{Grapheme_Cluster_Break=CR}\\p{Grapheme_Cluster_Break=LF}]': 1108169,
 '\\p{Grapheme_Cluster_Break=CR}': 1,
 '\\p{Grapheme_Cluster_Break=LF}': 1,
 '\\p{Grapheme_Cluster_Break=Control}': 3893,
 '\\p{Grapheme_Cluster_Break=Prepend}': 27,
 '\\p{Grapheme_Cluster_Break=L}': 125,
 '\\p{Grapheme_Cluster_Break=V}': 95,
 '\\p{Grapheme_Cluster_Break=LV}': 399,
 '\\p{Grapheme_Cluster_Break=LVT}': 10773,
 '\\p{Grapheme_Cluster_Break=T}': 137,
 '\\p{Grapheme_Cluster_Break=RI}': 26,
 '\\p{Extended_Pictographic}': 3537,
 '\\p{Grapheme_Cluster_Break=Extend}': 2130,
 '\\p{Grapheme_Cluster_Break=ZWJ}': 1,
 '\\p{InCB=Consonant}': 240,
 '\\p{InCB=Extend}': 884,
 '\\p{InCB=Linker}': 6,
 '\\p{Grapheme_Cluster_Break=SpacingMark}': 395}

In [28]:

def find_ranges(xss: list[bytes]):
    if not xss:
        return []
    xs = sorted([x[0] for x in set(xss)])
    ranges = []
    start = prev = xs[0]
    for x in xs[1:]:
        if prev + 1 != x:
            ranges.append(range(start, prev + 1)) # exclusive
            start = x
        prev = x
    else:
        ranges.append(range(start, prev + 1))
    return ranges

def format_ranges(ranges: list[range]):
    return b'[' + b''.join(
        bytes([range.start]) + b'-' + bytes([range.stop - 1])
        if len(range) > 2
        else bytes([range.start]) + bytes([range.stop - 1])
        if len(range) == 2
        else bytes([range.start])
        for range in ranges
    ) + b']'

x = find_ranges([b"a", b"c", b"b", b"e"])
x, format_ranges(x)

([range(97, 100), range(101, 102)], b'[a-ce]')

In [29]:
import re

def form_prefix_tree(*bins: bytes):
    prefix_tree = {}
    for bin in bins:
        if bin[0] in prefix_tree and bin[1:]:
            prefix_tree[bin[0]].append(bin[1:])
        elif bin[1:]:
            prefix_tree[bin[0]] = [bin[1:]]
        else:
            prefix_tree[bin[0]] = []
    # recurse
    return {i: form_prefix_tree(*cs) for i, cs in prefix_tree.items()}

def saturate(tree: dict):
    return {re.escape(bytes([key])): saturate(value) for key, value in tree.items()}

def prefix_to_regex(tree: dict):
    branches = [(key, prefix_to_regex(val)) for key, val in tree.items()]
    singles = [key for key, trail in branches if not trail and re.escape(key) == key]
    nonsingles = [key + trail for key, trail in branches if trail or re.escape(key) != key]
    single_ranges = format_ranges(find_ranges(singles))
    single_block = b"".join(singles)
    nonsingle_bundle = b'|'.join(nonsingles)
    if singles and nonsingles:
        return b"(?:" + single_ranges + b'|' + nonsingle_bundle + b")"
    elif nonsingles:
        if len(nonsingles) == 1:
            return nonsingle_bundle
        else:
            return b"(?:" + nonsingle_bundle + b")"
    elif singles:
        if len(singles) == 1:
            return single_block
        else:
            return single_ranges
    else:
        return b''

def prefix_to_regex_old(tree: dict):
    return b'(?:' + b'|'.join(key + prefix_to_regex_old(val) for key, val in tree.items()) + b')'

def chars_union(*cs: str):
    prefix_tree = form_prefix_tree(*(c.encode("utf-8") for c in cs))
    # print(prefix_tree)
    # print(saturate(prefix_tree))
    result = prefix_to_regex(saturate(prefix_tree))
    result_old = prefix_to_regex_old(saturate(prefix_tree))

    # print(result)
    # result = b"(?:" + b"|".join(regexify(c) for c in cs) + b")"
    # print(result)
    print("initial", len(result_old), "final", len(result), "delta", len(result) - len(result_old))
    return result

chars_union(*"abあいうえお+")

initial 62 final 21 delta -41


b'(?:[ab]|\xe3\x81[\x82\x84\x86\x88\x8a]|\\+)'

In [30]:
cat_ranges = {cat.encode("utf-8"): chars_union(*chars) for cat, chars in cat_chars.items()}
{cat: len(union) for cat, union in cat_ranges.items()}

initial 6737001 final 122703 delta -6614298
initial 10 final 2 delta -8
initial 10 final 2 delta -8
initial 23759 final 529 delta -23230
initial 255 final 73 delta -182
initial 778 final 30 delta -748
initial 603 final 41 delta -562
initial 3292 final 1120 delta -2172
initial 65536 final 2370 delta -63166
initial 850 final 30 delta -820
initial 174 final 8 delta -166
initial 21650 final 630 delta -21020
initial 13713 final 1501 delta -12212
initial 19 final 3 delta -16
initial 1503 final 109 delta -1394
initial 5962 final 796 delta -5166
initial 74 final 22 delta -52
initial 2828 final 650 delta -2178


{b'[^\\p{Grapheme_Cluster_Break=Control}\\p{Grapheme_Cluster_Break=CR}\\p{Grapheme_Cluster_Break=LF}]': 122703,
 b'\\p{Grapheme_Cluster_Break=CR}': 2,
 b'\\p{Grapheme_Cluster_Break=LF}': 2,
 b'\\p{Grapheme_Cluster_Break=Control}': 529,
 b'\\p{Grapheme_Cluster_Break=Prepend}': 73,
 b'\\p{Grapheme_Cluster_Break=L}': 30,
 b'\\p{Grapheme_Cluster_Break=V}': 41,
 b'\\p{Grapheme_Cluster_Break=LV}': 1120,
 b'\\p{Grapheme_Cluster_Break=LVT}': 2370,
 b'\\p{Grapheme_Cluster_Break=T}': 30,
 b'\\p{Grapheme_Cluster_Break=RI}': 8,
 b'\\p{Extended_Pictographic}': 630,
 b'\\p{Grapheme_Cluster_Break=Extend}': 1501,
 b'\\p{Grapheme_Cluster_Break=ZWJ}': 3,
 b'\\p{InCB=Consonant}': 109,
 b'\\p{InCB=Extend}': 796,
 b'\\p{InCB=Linker}': 22,
 b'\\p{Grapheme_Cluster_Break=SpacingMark}': 650}

In [31]:
bin_pattern = cat_pattern.encode("utf-8")
for cat, union in cat_ranges.items():
    bin_pattern = bin_pattern.replace(cat, union)

len(bin_pattern)

133892

In [32]:
import re, grapheme

final_pattern = re.compile(bin_pattern)
s = " 😀😀😀👨‍👩‍👧‍👦👨‍👩‍👧‍👦😀😀"
len(final_pattern.findall(s.encode("utf-8"))), grapheme.length(s)

(8, 8)

In [34]:
line_width = 79
offset = len("pattern = (")
indent = len("    ")
delimiters = len("b''")

bin_remaining = ascii(bin_pattern)[2:-1]

undecorated = []
remaining_budget = line_width - indent - delimiters
initial_budget = line_width - offset - delimiters
initial_budget = remaining_budget

i = 0
while i < len(bin_remaining):
    budget = initial_budget if i == 0 else remaining_budget

    nudge = 0
    attempt = bin_remaining[i : i + budget - nudge]
    while True:
        try:
            eval("b'" + attempt + "'")
        except SyntaxError:
            nudge += 1
            attempt = bin_remaining[i : i + budget - nudge]
        else:
            break
    
    undecorated.append((attempt, nudge))
    i += len(attempt)


undecorated

[('(?:\\\\\\r\\\\\\n|\\\\\\r|\\\\\\n)|(?:[\\x00-\\x08\\x0e-\\x1f\\x7f]|\\\\\\t|\\\\\\x0b|\\\\\\x0c|',
  2),
 ('\\xc2[\\x80-\\x9f\\xad]|\\xd8\\x9c|\\xe1\\xa0\\x8e|\\xe2(?:\\x80[\\x8b\\x8e\\x8f\\xa8-',
  1),
 ('\\xae]|\\x81[\\xa0-\\xaf])|\\xef(?:\\xbb\\xbf|\\xbf[\\xb0-\\xbb])|\\xf0(?:\\x93\\x90[',
  0),
 ('\\xb0-\\xbf]|\\x9b\\xb2[\\xa0-\\xa3]|\\x9d\\x85[\\xb3-\\xba])|\\xf3\\xa0(?:\\x80[\\x80',
  0),
 ('-\\x9f]|\\x82[\\x80-\\xbf]|\\x83[\\x80-\\xbf]|\\x87[\\xb0-\\xbf]|\\x88[\\x80-\\xbf]|',
  1),
 ('\\x89[\\x80-\\xbf]|\\x8a[\\x80-\\xbf]|\\x8b[\\x80-\\xbf]|\\x8c[\\x80-\\xbf]|\\x8d[',
  3),
 ('\\x80-\\xbf]|\\x8e[\\x80-\\xbf]|\\x8f[\\x80-\\xbf]|\\x90[\\x80-\\xbf]|\\x91[\\x80-',
  3),
 ('\\xbf]|\\x92[\\x80-\\xbf]|\\x93[\\x80-\\xbf]|\\x94[\\x80-\\xbf]|\\x95[\\x80-\\xbf]|',
  2),
 ('\\x96[\\x80-\\xbf]|\\x97[\\x80-\\xbf]|\\x98[\\x80-\\xbf]|\\x99[\\x80-\\xbf]|\\x9a[',
  3),
 ('\\x80-\\xbf]|\\x9b[\\x80-\\xbf]|\\x9c[\\x80-\\xbf]|\\x9d[\\x80-\\xbf]|\\x9e[\\x80-',
  3),
 ('\\xbf]|\\x9f[\\x80-\\xbf]|\\x

In [35]:
prefix = (
"""def length(string: bytes) -> int:
    '''Finds the length of a string'''
    import re
    return len(re.findall(pattern, string))

"""
)

with open("part.py", "wt") as f:
    f.write(prefix)
    f.write("pattern = (\n")
    for i, (line, nudge) in enumerate(undecorated):
        f.write("    b'")
        f.write(line)
        f.write("'")
        if nudge:
            f.write(nudge * "#")
        f.write("\n")
    f.write(")")

In [36]:
import part

part.length(" 😀😀😀👨‍👩‍👧‍👦👨‍👩‍👧‍👦😀😀".encode("utf-8"))

8